In [2]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from scipy import ndimage
from skimage import measure, color, io

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import joblib, json, sys, os
from IPython.display import Image, display_png
import json, glob, os.path

import tifffile
import pandas as pd
import sys, os, os.path, re, csv, math
import linecache
from statistics import mean,stdev
import seaborn as sns

import csv
import pprint
from statannotations.Annotator import Annotator

import skimage.morphology
import sklearn.mixture


# This code was developed by modifying the CUBIC-informatics code 
# from Matsumoto, Mitani, Horiguchi et al., Nature Protocols 2019 (https://github.com/lsb-riken/CUBIC-informatics).
# This analysis includes some unnecessary parameters, but default values are provided for compatibility purposes.

# Note: We are currently using local directory names for data paths (e.g., /export3/Imaging/...).
# Please replace these with your own file paths as needed.

depth=3
sigma_z=(1, 1)
sigma_xy=(8.0, 10.0)
radius_small=(24,3)
radius_large=(100,5)
gamma=1.0

class HDoG_CPU(object):
    def __init__(self, width=2048, height=2048, depth=None, sigma_xy=(4.0, 6.0), sigma_z=(1.8,2.7),
                 radius_small=(24,3), radius_large=(100,5), min_intensity=1000, gamma=1.0):
        self.width  = width
        self.height = height
        self.depth  = depth

        if type(sigma_xy) in [float,int]:
            self.sigma_xy = (sigma_xy, sigma_xy*1.5)
        else:
            self.sigma_xy = sigma_xy
        if type(sigma_z) in [float,int]:
            self.sigma_z = (sigma_z, sigma_z*1.5)
        else:
            self.sigma_z = sigma_z

        if not radius_small:
            self.radius_small_xy = int(self.sigma_xy[1]*4)
            self.radius_small_z = int(self.sigma_z[1]*2)
        else:
            self.radius_small_xy = radius_small[0]
            self.radius_small_z = radius_small[1]
        self.size_small = (self.radius_small_z*2+1, self.radius_small_xy*2+1, self.radius_small_xy*2+1)#self.radius_small_z*2+1

        if not radius_large:
            self.radius_large_xy = int(self.sigma_xy[1]*30)
            self.radius_large_xy = int(self.sigma_z[1]*10)
        else:
            self.radius_large_xy = radius_large[0]
            self.radius_large_z = radius_large[1]
        self.size_large = (self.radius_large_z*2+1, self.radius_large_xy*2+1, self.radius_large_xy*2+1)#self.radius_large_z*2+1

        self.min_intensity = min_intensity

        self.gamma = gamma
        self.normalizer = (self.sigma_xy[0]**(gamma*2)) * (self.sigma_z[0]**gamma)

    def load_images(self, list_images, dtype=np.uint16, height=2048, width=2060):
        # self.height, self.width, self.depthを書き換えた
        imgs = []
        for path in list_images:
            img = np.fromfile(path, dtype=dtype).reshape(height, width)
            imgs.append(img)
        imgs = np.array(imgs)
        depth = imgs.shape[0]
        return imgs

    def Normalize(self, src_img):
        print("Normalize start")
        dilation_l_img = scipy.ndimage.filters.uniform_filter(
            scipy.ndimage.morphology.grey_dilation(src_img, size=size_large, mode="nearest").astype(np.float32),
            size=size_large, mode="constant", cval=0)
        print("Dilation finish")
        erosion_l_img = scipy.ndimage.filters.uniform_filter(
            scipy.ndimage.morphology.grey_erosion(src_img, size=size_large, mode="nearest").astype(np.float32),
            size=size_large, mode="constant", cval=0)
        print("Erosion finish")

        intensity = src_img.astype(np.float32)
        #norm_img = ((intensity >= min_intensity) * intensity - erosion_l_img) / (dilation_l_img - erosion_l_img)
        norm_img = (intensity >= min_intensity) * intensity / (dilation_l_img - erosion_l_img)
        print("Normalize finish")
        return norm_img

    def CCL(self, pd_img):
        labels_img = skimage.morphology.label(pd_img)
        return labels_img

In [5]:
if type(sigma_xy) in [float,int]:
    sigma_xy = (sigma_xy, sigma_xy*1.5)
else:
    sigma_xy = sigma_xy
if type(sigma_z) in [float,int]:
    sigma_z = (sigma_z, sigma_z*1.5)
else:
    sigma_z = sigma_z

if not radius_small:
    radius_small_xy = int(sigma_xy[1]*4)
    radius_small_z = int(sigma_z[1]*2)
else:
    radius_small_xy = radius_small[0]
    radius_small_z = radius_small[1]
size_small = (radius_small_z*2+1, radius_small_xy*2+1, radius_small_xy*2+1)

if not radius_large:
    radius_large_xy = int(sigma_xy[1]*30)
    radius_large_xy = int(sigma_z[1]*10)
else:
    radius_large_xy = radius_large[0]
    radius_large_z = radius_large[1]
size_large = (radius_large_z*2+1, radius_large_xy*2+1, radius_large_xy*2+1)

#min_intensity = min_intensity

gamma = gamma
normalizer = (sigma_xy[0]**(gamma*2)) * (sigma_z[0]**gamma)

print(size_large)

(11, 201, 201)


In [8]:
# Let's begin the extraction of Tau deposition ROIs (regions of interest) using Gaussian-mean difference.
# For detailed information, please refer to the Materials and Methods section.

In [8]:
# 3D image path
# Note: We are currently using our local directory names (e.g., /export3/Imaging/...) for data paths.
# Please refer to the same name files uploaded on GitHub as needed.

# src_dir_SYTOXG: nuclear stained whole-brain data. This is needed for whole-brain registration to atlas. 
# src_dir_Tau: pTau whole-brain data.
## In this study, we utilized the CUBIC-Cloud registration software, a paid service.
## For those preferring a local setup, an alternative using ANTs registration with CUBIC-atlas 1.1 is available. For more information, refer to Mano et al., 2021.
## Detailed instructions for this alternative setup (excluding CUBIC-Cloud) are provided in this ipynb file.

# Note: This github does not currently include source data from light-sheet imaging.If you contact us via e-mail or github, we can share you.

PathPool=["Sahara_project/ID357_6.5_6.5_8.3um_version/"] 
dir1 = "/export3/Imaging/Axial/"+PathPool[0]

src_dir_SYTOXG = "ID357_SYTOX_Tiled_image_stack_8bit.tiff"
src_dir_Tau = "ID357_Tau_Tiled_image_stack_8bit_RP+.tiff"

Dir = "/export3/Imaging/Axial/"+PathPool[0]+"/" + src_dir_SYTOXG # SYTOXG
image_nuc = tifffile.imread(Dir)

Dir = "/export3/Imaging/Axial/"+PathPool[0]+"/" + src_dir_Tau # AT8
image_tau = tifffile.imread(Dir)

In [11]:
# z , y,  x
image_nuc.shape

(1068, 2560, 2160)

In [11]:
# 6.5x6.5x8.3um resolution → 8.3x8.3x8.3 um resolution
# Note: We are currently using local directory names for data paths (e.g., /export3/Imaging/...).
# Please replace these with your own file paths as needed.

resize_img = ndimage.zoom(image_nuc, (1.0, 6.5/8.3, 6.5/8.3), order = 0, mode='nearest')

dir1 = "/export3/Imaging/Axial/"+PathPool[0]

tifffile.imsave(
    dir1+"8.3um_isotropic_ID357_SYTOX_Tiled_image_stack_8bit.tiff",
    resize_img.astype(np.uint8)
)
print("OK")

resize_img = ndimage.zoom(image_tau , (1.0, 6.5/8.3, 6.5/8.3), order = 0, mode='nearest')

dir1 = "/export3/Imaging/Axial/"+PathPool[0]

tifffile.imsave(
    dir1+"8.3um_isotropic_ID357_Tau_Tiled_image_stack_8bit_RP+.tiff",
    resize_img.astype(np.uint8)
)
print("OK"

OK
OK


In [12]:
resize_img.shape

(1068, 2005, 1692)

In [67]:
# Read your resized images.

PathPool=["Sahara_project/ID357_6.5_6.5_8.3um_version/"] 
dir1 = "/export3/Imaging/Axial/"+PathPool[0]

src_dir_SYTOXG = "8.3um_isotropic_ID357_SYTOX_Tiled_image_stack_8bit.tiff"
src_dir_Tau = "8.3um_isotropic_ID357_Tau_Tiled_image_stack_8bit_RP+.tiff"

Dir = "/export3/Imaging/Axial/"+PathPool[0]+"/" + src_dir_SYTOXG # SYTOXG
image_nuc = tifffile.imread(Dir)

Dir = "/export3/Imaging/Axial/"+PathPool[0]+"/" + src_dir_Tau # AT8
image_tau = tifffile.imread(Dir)

In [69]:
# check
tifffile.imsave(
    dir1+"SYTOX-G_1.tif",
    image_nuc[500].astype(np.uint16)
)

tifffile.imsave(
    dir1+"Tau_1.tif",
    image_tau[500].astype(np.uint16)
)

In [11]:
# Normalize pTau images.

from scipy.ndimage import gaussian_filter
from scipy.ndimage import uniform_filter

size_large=(5,5,5)
min_intensity=0
image_normalized=[]
image_normalized= HDoG_CPU.Normalize(image_normalized, image_tau)
print("OK")

dir1 = "/export3/Imaging/Axial/"+PathPool[0]

tifffile.imsave(
    dir1+"image_normalized.tif",
    image_normalized.astype(np.float32)
)
print("OK")

Normalize start
Dilation finish
Erosion finish


/home/mitani/my_env/lib/python3.6/site-packages/ipykernel_launcher.py:71: RuntimeWarning: divide by zero encountered in true_divide


Normalize finish
OK
OK


In [12]:
# Apply a Gaussian filter.
# The kernel size must be optimized based on your image resolutions.
# Remarks: To achieve finer segmentation of images processed by Gaussian-mean difference, 
# it is necessary to optimize the ratio of kernel sizes for both the Gaussian and Mean filters 
# according to the size of the objects you want to segment.
# It is recommended to iteratively adjust the kernel sizes and have them checked by an expert.

fsize = 5 
image_tau_gaussed = gaussian_filter(image_tau, sigma=fsize/6)
print("OK")

OK


In [13]:
# Apply a Mean filter.
# The kernel size must be optimized based on your image resolutions.
# Remarks: To achieve finer segmentation of images processed by Gaussian-mean difference, 
# it is necessary to optimize the ratio of kernel sizes for both the Gaussian and Mean filters 
# according to the size of the objects you want to segment.
# It is recommended to iteratively adjust the kernel sizes and have them checked by an expert.

image_tau_mean = uniform_filter(image_tau, size=fsize*2)
print("OK")

OK


In [14]:
# check
dir1 = "/export3/Imaging/Axial/"+PathPool[0]

tifffile.imsave(
    dir1+"image_tau_gaussed.tif",
    image_tau_gaussed.astype(np.uint16)
)
print("OK")

OK


In [15]:
# check
dir1 = "/export3/Imaging/Axial/"+PathPool[0]

tifffile.imsave(
    dir1+"image_tau_gaussed1.tif",
    image_tau_gaussed[500].astype(np.uint16)
)
print("OK")

OK


In [16]:
# check
tifffile.imsave(
    dir1+"image_tau_mean.tif",
    image_tau_mean.astype(np.uint16)
)

print("OK")


OK


In [17]:
# Note: The surface of the brain always exhibits higher values, so create a mask to address this.
# Set the threshold at 1.5 times the overall mean.

mask_thresh = np.average(image_tau) * 1.5
image_tau_brain_mask =  image_tau>mask_thresh
print("OK")

OK


In [18]:
mask_thresh

18.143814742468873

In [19]:
from skimage.filters import threshold_minimum,threshold_mean,threshold_sauvola

In [20]:
image_tau_bg_mask = image_tau<=mask_thresh
print("OK")

OK


In [21]:
# check
tifffile.imsave(
    dir1+"image_tau_brain_mask.tif",
    image_tau_brain_mask.astype(np.uint16)
)
print("OK")

tifffile.imsave(
    dir1+"image_tau_bg_mask.tif",
    image_tau_bg_mask.astype(np.uint16)
)

print("OK")

OK
OK


In [22]:
# Read brain mask
from scipy import ndimage
Dir = "/export3/Imaging/Axial/"+PathPool[0]+"/image_tau_brain_mask.tif"
image_tau_brain_mask = tifffile.imread(Dir)

In [23]:
# Do erosion
image_tau_brain_mask_erosed = ndimage.binary_erosion(image_tau_brain_mask, iterations=10).astype(image_tau_brain_mask.dtype)
print("OK")

OK


In [24]:
# check
dir1 = "/export3/Imaging/Axial/"+PathPool[0]
tifffile.imsave(
    dir1+"image_tau_brain_mask_erosed.tif",
    image_tau_brain_mask_erosed.astype(np.uint16)
)

print("OK")

OK


In [25]:
# check
dir1 = "/export3/Imaging/Axial/"+PathPool[0]
tifffile.imsave(
    dir1+"image_tau_brain_mask_erosed1.tif",
    image_tau_brain_mask_erosed[500].astype(np.uint16)
)

print("OK")

OK


In [26]:
# check
tifffile.imsave(
    dir1+"image_tau_brain_mask1.tif",
    image_tau_brain_mask[500].astype(np.uint16)
)
print("OK")

OK


In [27]:
# Read
Dir = "/export3/Imaging/Axial/"+PathPool[0]+"/image_tau_gaussed.tif"
image_tau_gaussed = tifffile.imread(Dir)
Dir = "/export3/Imaging/Axial/"+PathPool[0]+"/image_tau_mean.tif"
image_tau_mean = tifffile.imread(Dir)

In [28]:
# Using Gaussian-mean, retain only the positive values.
image_tau_diff = np.subtract(image_tau_gaussed.astype(np.float32),image_tau_mean.astype(np.float32))
image_tau_diff = np.where(image_tau_diff>0,  image_tau_diff, 0)
print("OK")

OK


In [29]:
# check
tifffile.imsave(
    dir1+"image_tau_diff.tif",
    image_tau_diff.astype(np.uint16)
)

print("OK")

OK


In [30]:
# Read
Dir = "/export3/Imaging/Axial/"+PathPool[0]+"/image_tau_diff.tif"
image_tau_diff = tifffile.imread(Dir)

tifffile.imsave(
    dir1+"image_tau_diff_erosed.tif",
    image_tau_diff*image_tau_brain_mask_erosed.astype(np.uint16)
)
print("OK")

OK


In [31]:
# check
tifffile.imsave(
    dir1+"image_tau_diff_erosed1.tif",
    (image_tau_diff*image_tau_brain_mask_erosed)[500].astype(np.uint16)
)
print("OK")

OK


In [32]:
# check
tifffile.imsave(
    dir1+"image_tau_diff1.tif",
    (image_tau_diff)[500].astype(np.uint16)
)
print("OK")

OK


In [33]:
# Read
Dir = "/export3/Imaging/Axial/"+PathPool[0]+"/image_tau_diff_erosed.tif"
image_tau_diff_erosed = tifffile.imread(Dir)

In [34]:
# check
tifffile.imsave(
    dir1+"image_tau_diff_erosed2.tif",
    (image_tau_diff_erosed)[500].astype(np.uint16)
)
print("OK")

OK


In [4]:
# Read
Dir = "/export3/Imaging/Axial/"+PathPool[0]+"/image_tau_bg_mask.tif"
image_tau_bg_mask = tifffile.imread(Dir)

In [36]:
# check
tifffile.imsave(
    dir1+"image_tau_bg_mask2.tif",
    (image_tau_bg_mask)[500].astype(np.uint16)
)
print("OK")

OK


In [37]:
# read src pTau data.
Dir = "/export3/Imaging/Axial/"+PathPool[0]+src_dir_Tau # AT8
image_tau = tifffile.imread(Dir)

In [38]:
# Determine the threshold

thresh = np.average(image_tau, weights=image_tau_bg_mask) 
thresh = thresh * 2
print (thresh)
image_tau_thresh = image_tau_diff_erosed>thresh
#image_tau_thresh = np.where(image_tau_diff>thresh, 1, 0)
print("OK")

4.645829448598043
OK


In [39]:
# check
tifffile.imsave(
    dir1+"image_tau_thresh.tif",
    image_tau_thresh.astype(np.uint16)
)

print("OK")

OK


In [40]:
# check
tifffile.imsave(
    dir1+"image_tau_thresh1.tif",
    image_tau_thresh[500].astype(np.uint16)
)

print("OK")

OK


In [41]:
Dir = "/export3/Imaging/Axial/"+PathPool[0]+"/image_tau_thresh.tif"
image_tau_thresh = tifffile.imread(Dir)

In [42]:
# Label each ROI within the pTau depositions.
image_tau_CCLed=[]
image_tau_CCLed=HDoG_CPU.CCL(image_tau_CCLed, image_tau_thresh)

In [43]:
#  check
tifffile.imsave(
    dir1+"image_tau_CCLed1.tif",
    image_tau_CCLed[500].astype(np.float32)
)

print("OK")

OK


In [44]:
# check
tifffile.imsave(
    dir1+"image_tau_CCLed.tif",
    image_tau_CCLed.astype(np.float32)
)

print("OK")

OK


In [ ]:
# For each label, calculate the centroid and volume.

In [10]:
Dir = "/export3/Imaging/Axial/"+PathPool[0]+"/image_tau_CCLed.tif"
image_tau_CCLed = tifffile.imread(Dir)

Dir = "/export3/Imaging/Axial/"+PathPool[0]+"/image_tau_diff.tif"
image_tau_diff = tifffile.imread(Dir)

In [11]:
norm_img = image_tau_diff
labels_img = image_tau_CCLed.astype(np.int64)
on_region = np.nonzero(labels_img)
labels_list = labels_img[on_region]
num_labels = int(np.max(labels_list))
print(num_labels)

204087


In [12]:
norm_img.shape

(1121, 2560, 1950)

In [13]:
labels_img.shape

(1121, 2560, 1950)

In [14]:
labels_list

array([     1,      1,      1, ..., 204085, 204086, 204086])

In [15]:
# Sum of intensity
sum_normalized = scipy.ndimage.sum(norm_img, labels=labels_img, index=range(1, num_labels+1))
# region size
ns = np.ones(len(labels_list))

In [16]:
region_size = np.bincount(labels_list.astype(np.int64)-1, weights=ns)

In [17]:
df = pd.DataFrame({
    "index": pd.Series(np.arange(num_labels)),
    "intensity": pd.Series(sum_normalized),
    "size": pd.Series(region_size)
})

In [18]:
df

,index,intensity,size
0,0,9.0,4.0
1,1,33.0,13.0
2,2,7.0,3.0
3,3,29.0,12.0
4,4,39.0,13.0
...,...,...,...
204082,204082,6.0,3.0
204083,204083,18.0,8.0
204084,204084,4.0,2.0
204085,204085,8.0,4.0


In [19]:
# Save as csv
# But it still lacks xyz information.
dir1 = "/export3/Imaging/Axial/"+PathPool[0]

csv_tau = dir1 + "csv_tau1.csv"

df= df.astype('int')
df.to_csv(csv_tau, index=False, header=True, chunksize=50000,
            columns=["index","intensity","size"])

In [10]:
Dir = "/export3/Imaging/Axial/"+PathPool[0]+"/image_tau_CCLed.tif"
image_tau_CCLed = tifffile.imread(Dir)

Dir = "/export3/Imaging/Axial/"+PathPool[0]+"/image_tau_diff.tif"
image_tau_diff = tifffile.imread(Dir)

In [11]:
labels_img = image_tau_CCLed.astype(np.int64)    
on_region = np.nonzero(labels_img)
labels_list = labels_img[on_region]
num_labels = int(np.max(labels_list))
print(num_labels)

274859


In [12]:
depth=image_tau_CCLed.shape[0]
width=image_tau_CCLed.shape[1]
height=image_tau_CCLed.shape[2]

In [13]:
height

1692

In [14]:
zgrid,ygrid,xgrid = np.mgrid[0:depth, 0:width, 0:height]

In [15]:
# Save Grids
# To conserve memory, data was saved. This is important as pTau data can sometimes be large.

In [15]:
dir1 = "/export3/Imaging/Axial/"+PathPool[0]

tifffile.imsave(
    dir1+"zgrid.tif",
    zgrid.astype(np.uint16)
)

print("OK")

OK


In [16]:
dir1 = "/export3/Imaging/Axial/"+PathPool[0]

tifffile.imsave(
    dir1+"ygrid.tif",
    ygrid.astype(np.uint16)
)

print("OK")

OK


In [17]:
dir1 = "/export3/Imaging/Axial/"+PathPool[0]

tifffile.imsave(
    dir1+"xgrid.tif",
    xgrid.astype(np.uint16)
)

print("OK")

OK


In [12]:
Dir = "/export3/Imaging/Axial/"+PathPool[0]+"/zgrid.tif"
zgrid = tifffile.imread(Dir).astype(np.uint16)
print("OK")
Dir = "/export3/Imaging/Axial/"+PathPool[0]+"/ygrid.tif"
ygrid = tifffile.imread(Dir).astype(np.uint16)
print("OK")
Dir = "/export3/Imaging/Axial/"+PathPool[0]+"/xgrid.tif"
xgrid = tifffile.imread(Dir).astype(np.uint16)
print("OK")

OK
OK
OK


In [13]:
zgrid.shape

(1068, 2005, 1692)

In [14]:
ygrid.shape

(1068, 2005, 1692)

In [15]:
xgrid.shape

(1068, 2005, 1692)

In [16]:
Dir = "/export3/Imaging/Axial/"+PathPool[0]+"/image_tau_CCLed.tif"
labels_img = tifffile.imread(Dir).astype(np.int64)

In [17]:
labels_img.shape

(1068, 2005, 1692)

In [21]:
dir1 = "/export3/Imaging/Axial/"+PathPool[0]
data_1 = pd.read_csv(filepath_or_buffer=dir1 + "csv_tau1.csv", encoding="ms932", sep=",")

In [22]:
data_1

,index,intensity,size
0,0,8,1
1,1,26,4
2,2,10,2
3,3,17,3
4,4,46,8
...,...,...,...
119474,119474,594,48
119475,119475,383,23
119476,119476,678,65
119477,119477,1121,70


In [23]:
region_size=np.array(data_1["size"])
labels_img_flatten=labels_img.flatten()

KeyboardInterrupt: 

In [21]:
# Calculate centrids of each pTau deposition ROI.
centroid_x = np.bincount(labels_img_flatten, weights=xgrid.flatten())[1:] / region_size
print("OK")
centroid_y = np.bincount(labels_img_flatten, weights=ygrid.flatten())[1:] / region_size
print("OK")
centroid_z = np.bincount(labels_img_flatten, weights=zgrid.flatten())[1:] / region_size
print("OK")

OK
OK
OK


In [33]:
index1=np.array(data_1["index"])

In [34]:
df = pd.DataFrame({
    "index":  index1,
    "centroid_x": pd.Series(centroid_x),
    "centroid_y": pd.Series(centroid_y),
    "centroid_z": pd.Series(centroid_z),
})

NameError: name 'centroid_x' is not defined

In [24]:
df

,index,centroid_x,centroid_y,centroid_z
0,0,1074.400000,1143.400000,30.400000
1,1,1108.250000,1198.750000,32.500000
2,2,1023.052632,1282.789474,33.105263
3,3,1050.000000,1268.000000,33.000000
4,4,1029.753968,1253.888889,36.420635
...,...,...,...,...
274854,274854,683.714286,766.285714,1056.285714
274855,274855,673.000000,721.666667,1057.000000
274856,274856,676.000000,752.000000,1057.000000
274857,274857,681.000000,760.000000,1057.000000


In [25]:
# save as csv
dir1 = "/export3/Imaging/Axial/"+PathPool[0]

csv_tau = dir1 + "csv_tau2.csv"

df= df.astype('int')
df.to_csv(csv_tau, index=False, header=True, chunksize=50000,
            columns=["index","centroid_x","centroid_y","centroid_z"])

KeyError: "Passing list-likes to .loc or [] with any missing labels is no longer supported. The following labels were missing: Index(['centroid_x', 'centroid_y', 'centroid_z'], dtype='object'). See https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike"

In [20]:
data_1 = pd.read_csv(filepath_or_buffer=dir1 + "csv_tau1.csv", encoding="ms932", sep=",")
data_2 = pd.read_csv(filepath_or_buffer=dir1 + "csv_tau2.csv", encoding="ms932", sep=",")

In [21]:
data_2

,index,centroid_x,centroid_y,centroid_z
0,0,1125,1370,30
1,1,1088,1396,31
2,2,1100,1402,31
3,3,1216,1425,33
4,4,1126,1331,33
...,...,...,...,...
204082,204082,728,1083,1096
204083,204083,734,1043,1098
204084,204084,703,1068,1098
204085,204085,725,1037,1099


In [22]:
blunk_array_0 =np.full(len(data_2["centroid_x"]), 0.0)

In [23]:
# Save in a CSV format compatible with CUBIC-Cloud.
# Fill in any unnecessary data with dummy data.

df12 = []
df12 = pd.DataFrame({
    "X":pd.Series(data_2["centroid_x"], dtype=np.int64),
    "Y":pd.Series(data_2["centroid_y"], dtype=np.int64),
    "Z":pd.Series(data_2["centroid_z"], dtype=np.int64),
    "deltaI":pd.Series(data_1["intensity"], dtype=np.int64),
    "BG":pd.Series(blunk_array_0, dtype=np.int64),
    "vol":pd.Series(data_1["size"], dtype=np.int64),
    "deltaI_total":pd.Series(data_1["intensity"], dtype=np.int64)
        })

In [24]:
df12

,X,Y,Z,deltaI,BG,vol,deltaI_total
0,1125,1370,30,9,0,4,9
1,1088,1396,31,33,0,13,33
2,1100,1402,31,7,0,3,7
3,1216,1425,33,29,0,12,29
4,1126,1331,33,39,0,13,39
...,...,...,...,...,...,...,...
204082,728,1083,1096,6,0,3,6
204083,734,1043,1098,18,0,8,18
204084,703,1068,1098,4,0,2,4
204085,725,1037,1099,8,0,4,8


In [25]:
df12["X"].max()

1720

In [26]:
df12["Y"].max()

2420

In [27]:
df12["Z"].max()

1099

In [28]:
dir1 = "/export3/Imaging/Axial/"+PathPool[0]

csv_tau = dir1 + "csv_tau12.csv"

df12= df12.astype('int')
df12.to_csv(csv_tau, index=False, header=True, chunksize=50000,
            columns=["X","Y","Z","deltaI","BG","vol","deltaI_total"])

In [29]:
# Convert to micrometers (um), compatible with CUBIC-Cloud.
scale = 8.3 #um

df13 = []
df13 = pd.DataFrame({
    "X":pd.Series(data_2["centroid_x"]*scale, dtype=np.int64),
    "Y":pd.Series(data_2["centroid_y"]*scale, dtype=np.int64),
    "Z":pd.Series(data_2["centroid_z"]*scale, dtype=np.int64),
    "deltaI":pd.Series(data_1["intensity"], dtype=np.int64),
    "BG":pd.Series(blunk_array_0, dtype=np.int64),
    "vol":pd.Series(data_1["size"]*scale*scale*scale, dtype=np.int64),
    "deltaI_total":pd.Series(data_1["intensity"], dtype=np.int64)
        })


dir1 = "/export3/Imaging/Axial/"+PathPool[0]

csv_tau = dir1 + "csv_tau12_um.csv"

df13= df13.astype('int')
df13.to_csv(csv_tau, index=False, header=True, chunksize=50000,
            columns=["X","Y","Z","deltaI","BG","vol","deltaI_total"])

In [42]:
df13

,X,Y,Z,deltaI,BG,vol,deltaI_total
0,5386,9810,182,8,0,571,8
1,5096,8930,215,26,0,2287,26
2,5212,8881,224,10,0,1143,10
3,5909,9437,224,17,0,1715,17
4,4863,8632,232,46,0,4574,46
...,...,...,...,...,...,...,...
119474,5129,5893,8905,594,0,27445,594
119475,4365,6631,8914,383,0,13151,383
119476,4565,6332,8930,678,0,37166,678
119477,4623,5909,8939,1121,0,40025,1121


In [39]:
# check
scale=8.3 # um
scale=scale/50 # 50um scaleに変換

ptsw25 = []
ptsw25 = pd.DataFrame({
    "x":pd.Series(np.array(df12["X"])*scale, dtype=np.float),
    "y":pd.Series(np.array(df12["Y"])*scale, dtype=np.float),
    "z":pd.Series(np.array(df12["Z"])*scale, dtype=np.float),
        })

In [40]:
ptsw25

,x,y,z
0,178.284,189.738,4.980
1,183.928,198.868,5.312
2,169.818,212.812,5.478
3,174.300,210.488,5.478
4,170.814,207.998,5.976
...,...,...,...
274854,113.378,127.156,175.296
274855,111.718,119.686,175.462
274856,112.216,124.832,175.462
274857,113.046,126.160,175.462


In [42]:
# check the pTau density.
depth_ori = int(1068*scale)
height_ori =int(2005*scale)
width_ori = int(1692*scale)

img_N_ori,_ = np.histogramdd(
    np.vstack([
        ptsw25['z'].values,
        ptsw25['y'].values,
        ptsw25['x'].values
        ]).T,
        bins=(depth_ori, height_ori, width_ori),
        range=[(0,depth_ori-1),(0,height_ori-1),(0,width_ori-1)]
    )
        

In [43]:
tifffile.imsave(
        dir1+ "tau_density_50um.tif",
        img_N_ori.astype(np.uint16)
    )

In [ ]:
# Please upload the files `tau_density_50um.tif` and `csv_tau12_um.csv` to CUBIC-Cloud.
# The guidelines of CUBIC-Cloud were detailed in Mano T et al., CellRepMethods, 2021 (DOIhttps://doi.org/10.1016/j.crmeth.2021.100038).
# Website URL: https://cubic-cloud.com/ 
# You can use the outputs from CUBIC-Cloud for further analysis, including the "Tau gradient consensus analysis.ipynb".

## CUBIC-Cloud effectively represents all pTau depositions as point clouds.
## Supplementary Figures 4D, 4E, and 5AB were generated using the CUBIC-Cloud platform.

In [121]:
## Registration without CUBIC-Cloud can be performed using the same parameters intended for CUBIC-Cloud.
## To do this, you have to install the ANTs software and set up the environment. 
# For guidance, refer to Matsumoto et al., 2019, and Mano et al., 2021.
# CUBIC atlas 1.1 data (tif and csv.gz)can be downloaded from CUBIC-Atlas and CUBIC-Cloud data website: http://cubic-atlas.riken.jp/

import json, os.path, os, re, time
import tifffile
import joblib
from docopt import docopt
import subprocess as sp
import pandas as pd
import numpy as np
import nibabel as nib
import scipy.spatial

def run_antsRegistration(prefix_ants, atlas_file, moving_file, dst_dir, threads):
    
    cmd = "ITK_GLOBAL_DEFAULT_NUMBER_OF_THREADS={THREADS} && "
    cmd += "{EXECUTABLE} -d 3 "
    cmd += "--initial-moving-transform [{ATLAS_FILE},{MOVING_FILE},1] "
    cmd += "--interpolation Linear "
    cmd += "--use-histogram-matching 0 "
    cmd += "--winsorize-image-intensities [0.05,1.0] "  
    cmd += "--float 0 "
    cmd += "--output [{DST_PREFIX},{WARPED_FILE},{INVWARPED_FILE}] "
    cmd += "--transform Affine[0.1] --metric MI[{ATLAS_FILE},{MOVING_FILE},1,128,Regular,0.5] --convergence [1000x1000x10000,1e-5,15] --shrink-factors 8x4x2 --smoothing-sigmas 3x2x1vox "
    cmd += "--transform SyN[0.1,3.0,0.0] --metric CC[{ATLAS_FILE},{MOVING_FILE},1,4] --convergence [500x500x500x50,1e-6,10] --shrink-factors 8x4x2x1 --smoothing-sigmas 3x2x1x0vox"
    cmd = cmd.format(
        THREADS = threads,
        EXECUTABLE = os.path.join(prefix_ants, "antsRegistration"),
        DST_PREFIX = os.path.join(dst_dir, "F2M_"),
        WARPED_FILE = os.path.join(dst_dir, "F2M_Warped.nii.gz"),
        INVWARPED_FILE = os.path.join(dst_dir, "F2M_InvWarped.nii.gz"),
        ATLAS_FILE = atlas_file,
        MOVING_FILE = moving_file,
    )
    print("[*] Executing : {}".format(cmd))
    sp.call(cmd, shell=True)
    return

def register(atlas_basedir, merging_basedir, mapping_basedir,
             prefix_ANTs, atlas_voxel_unit, sample_base_name, num_cpus=36,
             atlas_basename="iso_80um"):
    atlas_tif_path = os.path.join(atlas_basedir, "{}.tif".format(atlas_basename))
    atlas_nii_path = os.path.join(atlas_basedir, "{}.nii.gz".format(atlas_basename))
    moving_tif_path = os.path.join(merging_basedir, sample_base_name + ".tif")
    moving_nii_path = os.path.join(mapping_basedir, "whole.nii.gz")

    # prepare nifti image for atlas
    print("[*] Preparing nifti image for atlas...")
    img_atlas = tifffile.imread(atlas_tif_path)
    if not os.path.exists(atlas_nii_path):
        nii_atlas = nib.Nifti1Image(np.swapaxes(img_atlas,0,2), affine=None)
        aff = np.diag([-atlas_voxel_unit,-atlas_voxel_unit,atlas_voxel_unit,1])
        nii_atlas.header.set_qform(aff, code=2)
        nii_atlas.to_filename(atlas_nii_path)
    # prepare nifti image for moving
    print("[*] Preparing nifti image for moving...")
    img_moving = tifffile.imread(moving_tif_path)
    nii_moving = nib.Nifti1Image(np.swapaxes(img_moving,0,2), affine=None)
    aff = np.diag([-atlas_voxel_unit,-atlas_voxel_unit,atlas_voxel_unit,1])
    nii_moving.header.set_qform(aff, code=2)
    nii_moving.to_filename(movingn nii_path)

    # run registration
    run_antsRegistration(prefix_ants = prefix_ANTs,
                         atlas_file = atlas_nii_path,
                         moving_file = moving_nii_path,
                         dst_dir = mapping_basedir,
                         threads = num_cpus)
    return


register("/export3/Imaging/cubic-atlas_v1-1/cubic-atlas/",
         dir1,
         dir1,
         "/usr/local/ants-2.1.0-redhat/",
         80.0,
         "8.3um_to_50um_resized_rescaled16bit_isotropic_ID357_SYTOX_Tiled_image_stack_8bit-1",
         40,
         "iso_80um",       
)

[*] Preparing nifti image for atlas...
[*] Preparing nifti image for moving...
[*] Executing : ITK_GLOBAL_DEFAULT_NUMBER_OF_THREADS=40 && /usr/local/ants-2.1.0-redhat/antsRegistration -d 3 --initial-moving-transform [/export3/Imaging/cubic-atlas_v1-1/cubic-atlas/iso_80um.nii.gz,/export3/Imaging/Axial/Sahara_project/ID357_6.5_6.5_8.3um_version/whole.nii.gz,1] --interpolation Linear --use-histogram-matching 0 --winsorize-image-intensities [0.05,1.0] --float 0 --output [/export3/Imaging/Axial/Sahara_project/ID357_6.5_6.5_8.3um_version/F2M_,/export3/Imaging/Axial/Sahara_project/ID357_6.5_6.5_8.3um_version/F2M_Warped.nii.gz,/export3/Imaging/Axial/Sahara_project/ID357_6.5_6.5_8.3um_version/F2M_InvWarped.nii.gz] --transform Affine[0.1] --metric MI[/export3/Imaging/cubic-atlas_v1-1/cubic-atlas/iso_80um.nii.gz,/export3/Imaging/Axial/Sahara_project/ID357_6.5_6.5_8.3um_version/whole.nii.gz,1,128,Regular,0.5] --convergence [1000x1000x10000,1e-5,15] --shrink-factors 8x4x2 --smoothing-sigmas 3x2x1vo

KeyboardInterrupt: 

In [177]:
# Doing point transfer into atlas space

dt_annotated = np.dtype([
    ('X', 'f4'), ('Y', 'f4'), ('Z', 'f4'),
    ('mapped_X', 'f4'), ('mapped_Y', 'f4'), ('mapped_Z', 'f4'),
    ('Allen_ID', 'u2'), ('deltaI','u2'),('vol','u2')
])

def run_antsApplyTransformsToPoints(prefix_ants, src_csv, dst_csv, dst_dir):
    cmd = "{EXECUTABLE} "
    cmd += "-d 3 "
    cmd += "-i {SRC_CSV} "
    cmd += "-o {DST_CSV} "
    cmd += "-t [{AFFINE_MAT},1] "
    cmd += "-t {INVWARP_NII}"
    cmd = cmd.format(
        EXECUTABLE = os.path.join(prefix_ants, "antsApplyTransformsToPoints"),
        AFFINE_MAT = os.path.join(dst_dir, "F2M_0GenericAffine.mat"),
        INVWARP_NII = os.path.join(dst_dir, "F2M_1InverseWarp.nii.gz"),
        SRC_CSV = src_csv,
        DST_CSV = dst_csv,
    )
    print("[*] Executing : {}".format(cmd))
    # supress output
    with open(os.devnull, 'w') as devnull:
        sp.check_call(cmd, shell=True, stdout=devnull)
    return

def map_and_annotate_cellstacks(src_csv_path, annotated_csv_path, 
                                prefix_ANTs, mapping_basedir, atlas_points_path,
                                downscale_unit, max_distance):
    
    if atlas_points_path.endswith("pkl"):
        df_atlas = joblib.load(atlas_points_path)
    elif atlas_points_path.endswith("csv") or atlas_points_path.endswith("csv.gz"):
        df_atlas = pd.read_csv(atlas_points_path, skiprows=1, header=None,
                               names=["X(um)","Y(um)","Z(um)","atlasID"],
                               dtype={"X(um)":np.float32, "Y(um)":np.float32, "Z(um)":np.float32, "atlasID":np.uint16})
    df_src = pd.read_csv(
        src_csv_path + "csv_tau12_um.csv",
        dtype={"X":np.uint16, "Y":np.uint16, "Z":np.uint16,
              "deltaI":np.uint16, "vol":np.uint16}
    )
    
    # XYZの各列に50/80を掛ける
    df_src2=df_src.copy()
    df_src2["X"] = df_src["X"] * (80/50)
    df_src2["Y"] = df_src["Y"] * (80/50)
    df_src2["Z"] = df_src["Z"] * (80/50)
    
    df_src2.to_csv(src_csv_path + "csv_tau12_um_50_80_scale.csv", index=False, header=True, chunksize=50000,
                columns=["X","Y","Z","deltaI","BG","vol","deltaI_total"])
    
    run_antsApplyTransformsToPoints(
    prefix_ants = prefix_ANTs,
    src_csv = src_csv_path + "csv_tau12_um_50_80_scale.csv", # "csv_tau12_um.csv",
    dst_csv = annotated_csv_path + "tmp_csv_tau12_um_aligned_with_ID_0.csv",
    dst_dir = mapping_basedir)
    
    df_transformed = pd.read_csv(
        annotated_csv_path + "tmp_csv_tau12_um_aligned_with_ID_0.csv",
        dtype={"X":np.float32, "Y":np.float32, "Z":np.float32}
    )
    print("Start annotation")
    
    tree = scipy.spatial.cKDTree( np.array([
        df_atlas["X(um)"].values,
        df_atlas["Y(um)"].values,
        df_atlas["Z(um)"].values
    ]).T )
    
    
    dist, idx = tree.query( np.array([
            df_transformed["X"].values,
            df_transformed["Y"].values,
            df_transformed["Z"].values,
        ]).T, k=1, eps=0, p=2, distance_upper_bound=max_distance)
 
    atlas_ID = np.zeros(idx.shape)
    atlas_ID[idx != tree.n] = df_atlas["atlasID"].values[idx[idx != tree.n]]
    
    data_annotated =  np.zeros((len(df_transformed),), dtype=dt_annotated)
    
    data_annotated["X"] = df_src["X"]
    data_annotated["Y"] = df_src["Y"]
    data_annotated["Z"] = df_src["Z"]
    
    data_annotated["mapped_X"] = df_transformed["X"]#*(80/50)
    data_annotated["mapped_Y"] = df_transformed["Y"]#*(80/50)
    data_annotated["mapped_Z"] = df_transformed["Z"]#*(80/50)
    
    data_annotated["Allen_ID"] = atlas_ID
    data_annotated["deltaI"] = df_src["deltaI"]
    data_annotated["vol"] = df_src["vol"]
    
    data_annotated_df = pd.DataFrame(data_annotated)
     
    
    #joblib.dump(data_annotated, annotated_csv_path + "csv_tau12_um_aligned_with_ID_0.csv", compress=3)
    

    data_annotated_df.to_csv(annotated_csv_path + "csv_tau12_um_aligned_with_ID_0.csv", index=False, header=True, chunksize=50000 ,
                           columns=["X","Y","Z","mapped_X","mapped_Y","mapped_Z", "Allen_ID", "deltaI", "vol"])
    
    print("Annotation finished!")


map_and_annotate_cellstacks(
    dir1,
    dir1,
    "/usr/local/ants-2.1.0-redhat/",
    mapping_basedir =dir1,
    atlas_points_path = "/export3/Imaging/cubic-atlas_v1-1/cubic-atlas/CUBIC-Atlas.csv.gz",
    downscale_unit = 80.0,
    max_distance = 160.0,
)

[*] Executing : /usr/local/ants-2.1.0-redhat/antsApplyTransformsToPoints -d 3 -i /export3/Imaging/Axial/Sahara_project/ID357_6.5_6.5_8.3um_version/csv_tau12_um_50_80_scale.csv -o /export3/Imaging/Axial/Sahara_project/ID357_6.5_6.5_8.3um_version/tmp_csv_tau12_um_aligned_with_ID_0.csv -t [/export3/Imaging/Axial/Sahara_project/ID357_6.5_6.5_8.3um_version/F2M_0GenericAffine.mat,1] -t /export3/Imaging/Axial/Sahara_project/ID357_6.5_6.5_8.3um_version/F2M_1InverseWarp.nii.gz
Start annotation
Annotation finished!


In [55]:
df_atlas = pd.read_csv("/export3/Imaging/cubic-atlas_v1-1/cubic-atlas/CUBIC-Atlas.csv.gz")

In [122]:
df_atlas

,X(um),Y(um),Z(um),ABAID
0,11504.0,30496.7,10553.1,951.0
1,11640.3,30479.2,10529.0,951.0
2,11606.1,30480.1,10514.8,951.0
3,11635.3,30488.4,10509.6,951.0
4,11622.5,30495.2,10500.2,951.0
...,...,...,...,...
78620370,11868.0,5934.5,8701.6,68.0
78620371,11726.8,5964.2,8701.0,68.0
78620372,11995.9,5943.0,8679.0,68.0
78620373,12014.0,5952.5,8668.9,68.0


In [126]:
df_atlas["Y(um)"].max()

30844.6

In [118]:
30844.6/80

385.5575

In [17]:
# Note: We are currently using our local directory names (e.g., /export3/Imaging/...) for data paths.
# Please refer to the same name files uploaded on GitHub as needed.
# Read atlas information.Refer to Matsumoto Mitani Horiguchi et al 2019 Nature Protocols. (This file was uproaded in github.)

data_Atlas = pd.read_csv(filepath_or_buffer="/export3/Imaging/atlas_for_fig/count_all_rgb.csv", encoding="ms932", sep=",")

data_atlas=np.array(data_Atlas.values.tolist())

data_GO = pd.read_csv(filepath_or_buffer="/export3/Imaging/atlas_for_fig/GOvsID_6region_removed.csv", encoding="ms932", sep=",")

data_GO

,id,graph_order,R,G,B
0,997,0,255,255,255
1,68,7,38,143,69
2,667,8,38,143,69
3,52192,9,38,143,69
4,52196,10,38,143,69
...,...,...,...,...,...
666,129,1299,170,170,170
667,140,1300,170,170,170
668,145,1301,170,170,170
669,153,1302,170,170,170


In [18]:
# Remove signals related to the fiber tracts","VS"in this analysis.
# Exclude entries containing an Allen_ID in oya_id[0].

dir= "/export3/Imaging/CUBIC-Atlas-R-test/"
dir_ana=dir+"Allen_graph.json"

with open(dir_ana) as f:
    param_ana = json.load(f)
    
id=[]
path_id=[]

for el in param_ana:
    original = el['id']
    id.append(original)
    newid = el['structure_id_path']
    path_id.append(newid)

import ast
labels = []
ids = []
colorlist=[]
volumelist_53=[]

labels = ["fiber tracts","VS" ]
ids =    [1009,73]
colorlist=[(204/255, 204/255, 204/255),(170/255, 170/255, 170/255)]

cou=0
oya_id=[]

for i in ids:
    oya_id.append([])

for el in param_ana:
    temp_id = el['id']
    temp_path_id = el['structure_id_path']
    temp_oya_id = 997
    for ids_i in ids:     
        if temp_path_id.count(ids_i) > 0:
            temp_oya_id= ids_i 
    if temp_oya_id != 997 and temp_id in data_GO["id"].tolist():
        print(temp_id)
        oya_id[ids.index(temp_oya_id)].append(temp_id)
        cou = cou +1

1009
949
1016
665
538
900
848
916
336
117
125
832
62
158
911
93
229
794
798
1116
413
841
506
658
633
482
237
380
697
744
326
812
850
866
78
1123
553
728
48512
956
579
964
1108
971
48516
986
784
6
924
190
198
1092
48520
48524
102
1051
1060
1043
863
397
884
908
940
466
530
603
428
436
443
449
301
48528
54
349
753
690
681
673
802
595
611
81
98
108
129
140
145
153
164


In [14]:
df_tau = []
df_tau = pd.read_csv(
    dir1 + "csv_tau12_um_aligned_with_ID_0.csv",
)

In [15]:
df_tau

,X,Y,Z,mapped_X,mapped_Y,mapped_Z,Allen_ID,deltaI,vol
0,8914.0,9486.0,249.0,16767.8,18597.5,2495.98,1066,32,2858
1,9196.0,9943.0,265.0,17304.0,19331.1,2512.24,805,20,2287
2,8490.0,10640.0,273.0,16174.3,20621.2,2528.42,671,166,10863
3,8715.0,10524.0,273.0,16553.6,20392.3,2522.33,805,5,571
4,8540.0,10399.0,298.0,16213.1,20199.0,2474.91,965,1073,6509
...,...,...,...,...,...,...,...,...,...
274854,5668.0,6357.0,8764.0,10360.3,12168.4,17125.70,754,41,4002
274855,5585.0,5984.0,8773.0,10135.9,11480.5,16992.40,754,16,1715
274856,5610.0,6241.0,8773.0,10246.0,11954.6,17080.50,754,5,571
274857,5652.0,6308.0,8773.0,10322.4,12075.6,17127.30,754,5,571


In [22]:
df_tau_noise_removed

,X,Y,Z,mapped_X,mapped_Y,mapped_Z,Allen_ID,deltaI,vol
0,8914.0,9486.0,249.0,16767.8,18597.5,2495.98,1066,32,2858
1,9196.0,9943.0,265.0,17304.0,19331.1,2512.24,805,20,2287
2,8490.0,10640.0,273.0,16174.3,20621.2,2528.42,671,166,10863
3,8715.0,10524.0,273.0,16553.6,20392.3,2522.33,805,5,571
4,8540.0,10399.0,298.0,16213.1,20199.0,2474.91,965,1073,6509
...,...,...,...,...,...,...,...,...,...
274854,5668.0,6357.0,8764.0,10360.3,12168.4,17125.70,754,41,4002
274855,5585.0,5984.0,8773.0,10135.9,11480.5,16992.40,754,16,1715
274856,5610.0,6241.0,8773.0,10246.0,11954.6,17080.50,754,5,571
274857,5652.0,6308.0,8773.0,10322.4,12075.6,17127.30,754,5,571


In [32]:
# Remove signals related to the fiber tracts","VS"in this analysis.
# Exclude entries containing an Allen_ID in oya_id[0].

df_tau_noise_removed = []
df_tau_noise_removed = df_tau[~df_tau['Allen_ID'].isin(oya_id[0] + oya_id[1])]

In [24]:
blunk_array_0 =np.full(len(df_tau_noise_removed["X"]), 0.0)

In [33]:
df_tau_noise_removed

,X,Y,Z,mapped_X,mapped_Y,mapped_Z,Allen_ID,deltaI,vol
0,8914.0,9486.0,249.0,16767.8,18597.5,2495.98,1066,32,2858
1,9196.0,9943.0,265.0,17304.0,19331.1,2512.24,805,20,2287
2,8490.0,10640.0,273.0,16174.3,20621.2,2528.42,671,166,10863
3,8715.0,10524.0,273.0,16553.6,20392.3,2522.33,805,5,571
4,8540.0,10399.0,298.0,16213.1,20199.0,2474.91,965,1073,6509
...,...,...,...,...,...,...,...,...,...
274854,5668.0,6357.0,8764.0,10360.3,12168.4,17125.70,754,41,4002
274855,5585.0,5984.0,8773.0,10135.9,11480.5,16992.40,754,16,1715
274856,5610.0,6241.0,8773.0,10246.0,11954.6,17080.50,754,5,571
274857,5652.0,6308.0,8773.0,10322.4,12075.6,17127.30,754,5,571


In [37]:
df13

,X,Y,Z,deltaI,BG,vol,deltaI_total
0,8914,9486,249,32,0,2858,32
1,9196,9943,265,20,0,2287,20
2,8490,10640,273,166,0,10863,166
3,8715,10524,273,5,0,571,5
4,8540,10399,298,1073,0,6509,1073
...,...,...,...,...,...,...,...
252998,5668,6357,8764,41,0,4002,41
252999,5585,5984,8773,16,0,1715,16
253000,5610,6241,8773,5,0,571,5
253001,5652,6308,8773,5,0,571,5


In [40]:
# pTau data was saved as csv.
# This is equivalent to ID_357_tau_point_data_without_FT_VS.csv after removed "fiber tracts","VS" belonging smallest regions.

df13 = []
df13 = pd.DataFrame({
    "X":pd.Series(np.array(df_tau_noise_removed["mapped_X"]), dtype=np.int64),
    "Y":pd.Series(np.array(df_tau_noise_removed["mapped_Y"]), dtype=np.int64),
    "Z":pd.Series(np.array(df_tau_noise_removed["mapped_Z"]), dtype=np.int64),
    "Allen_ID":pd.Series(np.array(df_tau_noise_removed["Allen_ID"]), dtype=np.int64),
    "deltaI":pd.Series(np.array(df_tau_noise_removed["deltaI"]), dtype=np.int64),
    "BG":pd.Series(blunk_array_0, dtype=np.int64),
    "vol":pd.Series(np.array(df_tau_noise_removed["vol"]), dtype=np.int64),
    "deltaI_total":pd.Series(np.array(df_tau_noise_removed["deltaI"]), dtype=np.int64)
        })


dir1 = "/export3/Imaging/Axial/"+PathPool[0]

csv_tau = dir1 + "csv_tau12_um_aligned_FT_VS.csv"

df13= df13.astype('int')
df13.to_csv(csv_tau, index=False, header=True, chunksize=50000,
            columns=["X","Y","Z","Allen_ID","deltaI","BG","vol","deltaI_total"])

In [170]:
# Check: Tau Intensity per 50x50x50μm Voxel
ptsw25 = []
ptsw25 = pd.DataFrame({
    "x":pd.Series(np.array(df_tau_noise_removed["X"])*scale, dtype=np.float),
    "y":pd.Series(np.array(df_tau_noise_removed["Y"])*scale, dtype=np.float),
    "z":pd.Series(np.array(df_tau_noise_removed["Z"])*scale, dtype=np.float),
        })

depth_ori = 177#234
height_ori =333#413
width_ori = 281#335

img_N_ori,_ = np.histogramdd(
    np.vstack([
        ptsw25['z'].values,
        ptsw25['y'].values,
        ptsw25['x'].values
        ]).T,
        bins=(depth_ori, height_ori, width_ori),
        range=[(0,depth_ori-1),(0,height_ori-1),(0,width_ori-1)]
    )


tifffile.imsave(
        dir1+ "tau_density_50um_in_real_space.tif",
        img_N_ori.astype(np.uint16)
    )

In [183]:
# Check: Tau Intensity per 50x50x50μm Voxel
ptsw25 = []
ptsw25 = pd.DataFrame({
    "x":pd.Series(np.array(df_tau_noise_removed["mapped_X"])*scale, dtype=np.float),
    "y":pd.Series(np.array(df_tau_noise_removed["mapped_Y"])*scale, dtype=np.float),
    "z":pd.Series(np.array(df_tau_noise_removed["mapped_Z"])*scale, dtype=np.float),
        })

depth_ori = 234
height_ori =413
width_ori = 335

img_N_ori2,_ = np.histogramdd(
    np.vstack([
        ptsw25['z'].values,
        ptsw25['y'].values,
        ptsw25['x'].values
        ]).T,
        bins=(depth_ori, height_ori, width_ori),
        range=[(0,depth_ori-1),(0,height_ori-1),(0,width_ori-1)]
    )


tifffile.imsave(
        dir1+ "tau_density_50um_in_atlas_space2.tif",
        img_N_ori2.astype(np.uint16)
    )